# Final Notebook
# Created by: [Aaron Ooi, Viet Nguyen, Trevor Mathisen]

### Imports and setup

In [ ]:
final_models = [] # Add your final models here as a tuple of ('name', model)) for Part 3
%load_ext cuml.accel

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import tensorflow as tf

from numpy import ndarray, argsort, set_printoptions, argmax, isnan, nan, mean, random

from pandas import set_option, read_csv, DataFrame, concat, Series
from pandas.plotting import scatter_matrix

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, Normalizer
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, BaggingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, f1_score, roc_auc_score, mean_squared_error, r2_score
from sklearn.metrics import auc as auc_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from cuml.svm import SVC as cuSVC
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, roc_curve
from sklearn.metrics import auc as auc_score
import numpy as np
import matplotlib.pyplot as plt
import cudf
from cuml.metrics import accuracy_score
import cupy as cp
# Timing



# Part 1: EDA

In [ ]:
DONT_RUN_LONG_CELLS = True
kickstarter_filename = 'kickstarter_data_full.csv'
kickstarter_filename_features = 'kickstarter_data_with_features.csv'

ks_data = read_csv(kickstarter_filename)
ks_feat_data = read_csv(kickstarter_filename_features)
data_list = [('ks_data', ks_data), ('ks_feat_data', ks_feat_data)]

### Determine if the two datasets are the same

In [ ]:
# print columns not present in the other dataset
ks_data_columns = set(ks_data.columns)
ks_feat_data_columns = set(ks_feat_data.columns)

ks_data_not_in_feat = ks_data_columns - ks_feat_data_columns
ks_feat_data_not_in_ks = ks_feat_data_columns - ks_data_columns

print(f'Columns in ks_data not in ks_feat_data: {ks_data_not_in_feat}')
print(f'Columns in ks_feat_data not in ks_data: {ks_feat_data_not_in_ks}')

common_columns = ks_data_columns.intersection(ks_feat_data_columns)

# of common columns, compare the values and see if they match
for column in common_columns:
    ks_data_values = ks_data[column].unique()
    ks_feat_data_values = ks_feat_data[column].unique()
    if len(ks_data_values) != len(ks_feat_data_values):
        print(f'Column {column} has different number of unique values: {len(ks_data_values)} vs {len(ks_feat_data_values)}')
    else:
        pass

## So we know ks_data is the dataset to use as it has more data, more unique data


In [ ]:
data_list = [('ks_data', ks_data), ('ks_feat_data', ks_feat_data)]

def data_info(_data, show_scatter=False):
    print(f'Data head: {_data.head(5)}')
    print(f'Null values: {_data.isnull().sum()}')
    print(f'Data Shape: {_data.shape[0]} rows and {_data.shape[1]} columns')
    print(f'Columns: {list(_data.columns)}')
    for column in _data.columns:
        if len(_data[column].unique()) < 10:
            print(f'{column} unique values: {_data[column].unique()}')
        else:
            percent_unique = len(_data[column].unique()) / _data.shape[0] * 100
            print(f'{column} % unique values: {percent_unique}')
    print(_data.describe())
    _data.hist(figsize=(12, 10))
    plt.tight_layout()
    plt.show()
    plt.figure()  # new plot
    plt.tight_layout()

    # Only show for floats and ints

    float_columns = _data.select_dtypes(include=['float64']).columns
    int_columns = _data.select_dtypes(include=['int64']).columns
    # Combine float and int columns

    numeric_columns = float_columns.append(int_columns)
    # Calculate correlation matrix
    corMat = _data[numeric_columns].corr(method='pearson')

    print(corMat)
    ## plot correlation matrix as a heat map
    plt.figure(figsize=(14, 10))
    sns.heatmap(corMat, square=True)
    plt.yticks(rotation=0)
    plt.xticks(rotation=90)
    plt.title(f"CORRELATION MATRIX USING HEAT MAP")
    plt.show()

    ## scatter plot of all _data
    plt.figure()
    # # The output overlaps itself, resize it to display better (w padding)
    if show_scatter:
        try:
            scatter_matrix(_data)
            plt.tight_layout(pad=0.1)
            plt.show()
        except:
            return


def data_info_less(_date):
    set_option('display.max_columns', None)
    if not isinstance(_date, list):
        if not isinstance(_date, tuple):
            _date = ('', _date)
        _date = [_date]
    for name, data in _date:
        print(f'{name} data')
        print(data.info())
        print(data.head(5))
        for column in data.columns:
            highlight_column = 'profile'
            if column == highlight_column:
                # print 2 rows of values completely
                row1 = data.iloc[0][highlight_column]
                row2 = data.iloc[1][highlight_column]
                print(f'Row 1: {row1}')
                print(f'Row 2: {row2}')
            if len(data[column].unique()) < 10:
                print(f'{column} unique values: {data[column].unique()}')
            else:
                percent_unique = len(data[column].unique()) / data.shape[0] * 100
                print(f'{column} % unique values: {percent_unique}')
        break

In [ ]:
data_info(ks_data)


### By iteratively looking at columns in this cell, and looking at just those not categorized by a human, we can determine what columns are not needed, and types of others

### We also determine unique columns and their values to setup for one-hot encoding later

In [ ]:
columns_to_drop = ['Unnamed: 0', 'id', 'photo', 'name', 'blurb', 'slug', 'currency_symbol', 'currency_trailing_code', 'static_usd_rate', 'creator', 'profile', 'friends', 'is_backing', 'permissions', 'name_len', 'blurb_len', 'urls', 'source_url', 'location', 'is_starred', 'create_to_launch']
float_columns = ['goal', 'pledged', 'usd_pledged']
int_columns = ['backers_count', 'name_len_clean', 'blurb_len_clean',  'launch_to_deadline', 'launch_to_state_change', 'create_to_launch_days', 'launch_to_deadline_days', 'launch_to_state_change_days', ]
datetime_columns = ['deadline', 'state_changed_at', 'created_at', 'launched_at']
date_int_columns = ['deadline_month', 'deadline_day', 'deadline_hr', 'state_changed_at_month', 'state_changed_at_day', 'state_changed_at_month', 'state_changed_at_day', 'state_changed_at_yr', 'created_at_month', 'created_at_day', 'created_at_hr', 'launched_at_month', 'launched_at_day', 'launched_at_hr', 'state_changed_at_hr', 'created_at_yr', 'launched_at_yr']
category_columns = ['state', 'currency', 'staff_pick', 'category', 'deadline_weekday', 'state_changed_at_weekday', 'created_at_weekday', 'launched_at_weekday', 'deadline_yr', 'country']
boolean_columns = ['disable_communication', 'spotlight', 'SuccessfulBool', 'USorGB', 'TOPCOUNTRY', 'LaunchedTuesday', 'DeadlineWeekend']

# This is just to categorize everything and make sure I'm not missing anything
temp_data = ks_data.copy()
data_info_less(temp_data)
temp_data = temp_data.drop(columns=columns_to_drop)
temp_data = temp_data.drop(columns=datetime_columns)
temp_data = temp_data.drop(columns=category_columns)
temp_data = temp_data.drop(columns=boolean_columns)
temp_data = temp_data.drop(columns=float_columns)
temp_data = temp_data.drop(columns=int_columns)
temp_data = temp_data.drop(columns=date_int_columns)
print('\n' * 3)
print(f'Lets see what we have left')
data_info_less(('temp_data', temp_data))

### Examine and clean columns

In [ ]:
kickstarter = ks_data.copy()
#kickstarter = kickstarter.dropna()
kickstarter = kickstarter.drop(columns=columns_to_drop)
# Specifically examine the problematic columns
print("NaN in name_len_clean:", kickstarter['name_len_clean'].isna().sum())
print("NaN in blurb_len_clean:", kickstarter['blurb_len_clean'].isna().sum())

print(kickstarter['category'].unique())
# total number of records
print(kickstarter.shape)
# Number of na's in 'category'
category_nas = kickstarter['category'].isna().sum()
total_rows = kickstarter.shape[0]
print(f'Category has {(category_nas / total_rows * 100):.2f}% as "nan"')
temp_data = ks_data.copy()
# Drop the 10 rows which have _len_clean as nan
kickstarter = kickstarter.dropna(subset=['name_len_clean', 'blurb_len_clean'])
print("NaN in name_len_clean:", kickstarter['name_len_clean'].isna().sum())
print("NaN in blurb_len_clean:", kickstarter['blurb_len_clean'].isna().sum())

kickstarter['name_len_clean'] = kickstarter['name_len_clean'].astype(int)
kickstarter['blurb_len_clean'] = kickstarter['blurb_len_clean'].astype(int)

# Replace 'nan' in category with 'None' (string not object), then do one-hot encoding
kickstarter['category'] = kickstarter['category'].fillna("None")
print(kickstarter['category'].unique())

### One-hot encode the categorical columns

In [ ]:
def encode_categorical_features(df: DataFrame) -> tuple:
    """Encode all categorical features in the dataframe"""
    # Create a copy to avoid modifying the original
    df = df.copy()

    # Get all categorical columns
    categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
    print(f"Categorical columns: {categorical_columns}")

    # Handle boolean columns separately
    boolean_columns = ['staff_pick']
    for col in boolean_columns:
        if col in df.columns:
            df[col] = df[col].replace({False: 0, True: 1})
            print(f"Converted boolean column: {col}")

    # Columns to one-hot encode
    columns_to_encode = [
        'category',
        'deadline_weekday',
        'created_at_weekday',
        'launched_at_weekday'
    ]

    # Filter out columns that aren't in the dataframe
    columns_to_encode = [col for col in columns_to_encode if col in df.columns]

    # Remove 'state_changed_at_weekday' column if it exists
    if 'state_changed_at_weekday' in df.columns:
        df = df.drop(columns='state_changed_at_weekday')
        print("Dropped 'state_changed_at_weekday' column")

    # One-hot encode the selected columns
    all_encoded_dfs = []

    for col in columns_to_encode:
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        encoded = encoder.fit_transform(df[[col]])

        # Create meaningful column names
        encoded_cols = [f"{col}_{c}" for c in encoder.categories_[0]]
        encoded_df = DataFrame(encoded, columns=encoded_cols, index=df.index)

        # Add to list of encoded dataframes
        all_encoded_dfs.append(encoded_df)

        # Drop the original column
        print(f'Dropping original column: {col}')
        df = df.drop(columns=col)
        print(f"Encoded column: {col} → {len(encoded_cols)} features")

    # Combine all dataframes
    result = concat([df] + all_encoded_dfs, axis=1)

    # Create a list of all categorical columns (original and encoded)
    all_categorical_columns = []
    for col in categorical_columns:
        if col in result.columns:
            all_categorical_columns.append(col)
        else:
            # Add the encoded column names
            all_categorical_columns.extend([c for c in result.columns if c.startswith(f"{col}_")])

    print(f"Total categorical columns after encoding: {len(all_categorical_columns)}")

    return result, all_categorical_columns

# Example usage
kickstarter, category_columns = encode_categorical_features(kickstarter)

## Now we know what is in the data, let's look at some of the features and see if we can find any interesting patterns.

In [ ]:
# For each feature, lets see a histogram of the values and a boxplot of the values
def plot_feature_distribution(data, feature):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.histplot(data[feature], bins=30, kde=True)
    plt.title(f'{feature} Distribution')
    plt.xlabel(feature)
    plt.ylabel('Frequency')

    plt.subplot(1, 2, 2)
    sns.boxplot(x=data[feature])
    plt.title(f'{feature} Boxplot')
    plt.xlabel(feature)

    plt.tight_layout()
    plt.show()

columns_to_show = float_columns + int_columns

for feature in kickstarter.columns:
    if feature in columns_to_show and not DONT_RUN_LONG_CELLS:
        plot_feature_distribution(kickstarter, feature)

### Based on the above, perform scaling on the float and int columns

In [ ]:
columns_to_standardize = [ # Roughly normal
    'name_len_clean', 'blurb_len_clean',
    'deadline_month', 'deadline_day', 'deadline_hr',
    'state_changed_at_month', 'state_changed_at_day', 'state_changed_at_hr',
    'created_at_month', 'created_at_day', 'created_at_hr',
    'launched_at_month', 'launched_at_day', 'launched_at_hr'
]

columns_to_log_transform = [ # Heavily right skewed
    'goal', 'pledged', 'usd_pledged', 'backers_count',
    'create_to_launch_days'
]

columns_to_normalize = [ # Different scales
    'launch_to_deadline_days', 'launch_to_state_change_days'
]

columns_to_drop2 = [
    # Temporal columns represented by better features
    'deadline', 'state_changed_at', 'created_at', 'launched_at',
    'launch_to_deadline', 'launch_to_state_change',

    # Duplicate/redundant information
    'deadline_yr', 'state_changed_at_yr', 'created_at_yr', 'launched_at_yr',

    # Categorical with many unique values, better represented by other features
    'country', 'currency',

    # Low variance or binary columns that might be redundant
    'disable_communication', 'spotlight',
    'USorGB', 'TOPCOUNTRY', 'LaunchedTuesday', 'DeadlineWeekend'
]

# Drop the columns we don't want
kickstarter = kickstarter.drop(columns=columns_to_drop2)
print(kickstarter.columns)

for column in columns_to_log_transform:
    print(f"NaN values in '{column}': {kickstarter[column].isna().sum()}")

print(kickstarter.shape[0])
kickstarter = kickstarter.dropna(subset=columns_to_log_transform)
print(kickstarter.shape[0])

# Standardize the columns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for column in columns_to_standardize:
    kickstarter[column] = scaler.fit_transform(kickstarter[[column]])

# Log transform the columns
from sklearn.preprocessing import FunctionTransformer
import numpy as np
log_transformer = FunctionTransformer(np.log1p, validate=True)
for column in columns_to_log_transform:
    kickstarter[column] = log_transformer.fit_transform(kickstarter[[column]])
# Normalize the columns
from sklearn.preprocessing import MinMaxScaler
normalizer = MinMaxScaler()
for column in columns_to_normalize:
    kickstarter[column] = normalizer.fit_transform(kickstarter[[column]])

In [ ]:
# Check the data again
data_info(kickstarter)

### We need to make a 'success' classifier, so lets take a look at the definition of success in the data

In [ ]:
# Compare 'state' and 'SuccessfulBool' to see if they are the same
print(ks_data['state'].unique())
print(ks_data['SuccessfulBool'].unique())
print(ks_data['state'].value_counts())
print(ks_data['SuccessfulBool'].value_counts())
# Make a table where the rows are 0/1 for SuccessfulBool and the columns are the states
print(ks_data.groupby(['SuccessfulBool', 'state']).size().unstack())

# Show the distribution of 'pledged/goal' to see if we can make a column called 'got_funded'
temp_data = ks_data.copy()

# confirm they're both numbers
print(temp_data['pledged'].dtype)
print(temp_data['goal'].dtype)

# show the count where pledged > goal
print(f'{temp_data[temp_data['pledged'] > temp_data['goal']].shape[0]} projects got funded')

# Create got_funded column and show the histogram
temp_data['got_funded'] = temp_data['pledged'] / temp_data['goal']

# Describe the column
print(temp_data['got_funded'].describe())

# Get count of SuccessfulBool
print(temp_data['SuccessfulBool'].value_counts())
# Shows the same thing

# using kickstarter data, show some good plots of features vs SuccessfulBool

def plot_feature_vs_success(data, feature):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.histplot(data[data['SuccessfulBool'] == 1][feature], bins=30, kde=True, color='green', label='Successful')
    sns.histplot(data[data['SuccessfulBool'] == 0][feature], bins=30, kde=True, color='red', label='Failed')
    plt.title(f'{feature} vs SuccessfulBool')
    plt.xlabel(feature)
    plt.ylabel('Frequency')
    plt.legend()

    plt.subplot(1, 2, 2)
    sns.boxplot(x=data['SuccessfulBool'], y=data[feature])
    plt.title(f'{feature} vs SuccessfulBool')
    plt.xlabel('SuccessfulBool')
    plt.ylabel(feature)

    plt.tight_layout()
    plt.show()

# Show the features that are not categorical
for feature in kickstarter.columns:
    if DONT_RUN_LONG_CELLS:
        break
    if feature not in ['state', 'SuccessfulBool', 'category', 'currency', 'staff_pick']:
        plot_feature_vs_success(kickstarter, feature)

### Because `state` captures the same information as `SuccessfulBool`, we can drop it
### and because `got_funded` and `pledged` offer information obtained at the end of the campaign, we should drop them as well

In [ ]:
kickstarter = kickstarter.drop(columns=['state', 'pledged', 'usd_pledged'])
kickstarter['name_len_clean'] = kickstarter['name_len_clean'].astype(int)
kickstarter['blurb_len_clean'] = kickstarter['blurb_len_clean'].astype(int)

In [ ]:
kickstarter = kickstarter.sample(frac=1, random_state=42).reset_index(drop=True)

X = kickstarter.copy()
Y1 = kickstarter['SuccessfulBool']
X = X.drop(columns=['SuccessfulBool'])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, Y1, test_size=0.2, random_state=42, stratify=Y1)

# Part 2: Models (tuning and final models)

### Model 1: Random Forest Classifier

In [ ]:
def get_rf_feature_importance(_X_train: DataFrame, _y_train: ndarray, n_features: int = 10) -> tuple:
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(_X_train, _y_train)

    # Get feature importances
    importances = rf.feature_importances_
    indices = argsort(importances)[::-1]

    # Select top n features
    top_features = _X_train.columns[indices[:n_features]].tolist()
    return top_features, indices


def get_rfe_features(_X_train: DataFrame, _y_train: ndarray, n_features: int = 10) -> list:
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    rfe = RFE(estimator=model, n_features_to_select=n_features)
    rfe.fit(_X_train, _y_train)

    # Get selected features
    selected_features = _X_train.columns[rfe.support_].tolist()
    return selected_features

# Method 3: Statistical feature selection
def get_statistical_features(_X_train: DataFrame, _y_train: ndarray, n_features: int = 10) -> list:
    selector = SelectKBest(score_func=f_regression, k=n_features)
    selector.fit(_X_train, _y_train)

    # Get selected features
    selected_mask = selector.get_support()
    selected_features = _X_train.columns[selected_mask].tolist()
    return selected_features

# Get ordered features and indices from Random Forest
top_features, importance_indices = get_rf_feature_importance(X_train, y_train, len(X_train.columns))
ranked_features = X_train.columns[importance_indices].tolist()

# Define range of feature counts to test
feature_counts = [10, 20, 30, 40, 50, 60]
if len(ranked_features) < 30:
    feature_counts = list(range(5, len(ranked_features)+1, 5))

# Test different feature counts
cv_scores = []
for n_features in feature_counts:
    if n_features > len(ranked_features):
        continue

    # Select top n features
    selected_features = ranked_features[:n_features]
    X_selected = X_train[selected_features]

    # Evaluate with cross-validation
    scores = cross_val_score(
        RandomForestRegressor(n_estimators=100, random_state=42),
        X_selected, y_train,
        cv=5, scoring='r2'
    )

    cv_scores.append(scores.mean())
    print(f"{n_features} features: R² = {scores.mean():.4f}")
    print(f"Selected features: {selected_features}")

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(feature_counts[:len(cv_scores)], cv_scores, marker='o')
plt.xlabel('Number of Features')
plt.ylabel('Cross-Validated R² Score')
plt.title('Model Performance vs Number of Features')
plt.grid(True)
plt.savefig('feature_selection_curve.png')
plt.show()

# Find the optimal number based on the results
optimal_idx = max(range(len(cv_scores)), key=cv_scores.__getitem__)
optimal_n_features = feature_counts[optimal_idx]
print(f"\nOptimal number of features: {optimal_n_features}")
print(f"Selected features: {ranked_features[:optimal_n_features]}")


print(f"\nRandom Forest selected {optimal_n_features} features: {ranked_features[:optimal_n_features]}")


optimal_features = ranked_features[:optimal_n_features]

# Train the model using optimal features
X_train_optimal = X_train[optimal_features]
X_test_optimal = X_test[optimal_features]

# Define parameter grid for hyperparameter tuning
param_grid = {
    'max_depth': [None, 5, 10, 15, 20, 25, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create base model
base_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Setup Grid Search with cross-validation
grid_search = GridSearchCV(
    estimator=base_model,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,  # Use all available cores
    verbose=1
)

# Perform Grid Search
grid_search.fit(X_train_optimal, y_train)

# Get best parameters and best model
best_params = grid_search.best_params_
best_rf_model = grid_search.best_estimator_

print(f"Best Parameters: {best_params}")

# Make predictions with the best model
y_pred = best_rf_model.predict(X_test_optimal)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

# Feature importance of the final model
feature_importance = best_rf_model.feature_importances_
importance_df = DataFrame({
    'Feature': optimal_features,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

print("\nFeature Importance:")
print(importance_df.head(10))  # Show top 10 features

# Calculate cross-validation score for the best model
cv_scores = cross_val_score(
    best_rf_model,
    X[optimal_features], Y1,
    cv=5, scoring='accuracy'
)
print(f"\nCross-validation accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# Plot ROC curve
y_pred_proba = best_rf_model.predict_proba(X_test_optimal)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc_score(fpr, tpr)

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2,
         label=f'ROC curve (area = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Visualize hyperparameter tuning results
results = DataFrame(grid_search.cv_results_)

# Plot accuracy for different max_depth values
plt.figure(figsize=(12, 6))

# Filter results for default min_samples values
default_samples = results[(results['param_min_samples_split'] == 2) &
                         (results['param_min_samples_leaf'] == 1)]

# Convert None to -1 for plotting purposes
default_samples['param_max_depth'] = default_samples['param_max_depth'].fillna(-1)

# Sort by max_depth for proper x-axis order
default_samples = default_samples.sort_values('param_max_depth')

plt.plot(default_samples['param_max_depth'],
         default_samples['mean_test_score'],
         marker='o', linestyle='-')

# Replace -1 with "None" in xticks
x_ticks = default_samples['param_max_depth'].unique()
x_labels = [str(int(x)) if x > 0 else "None" for x in x_ticks]

plt.xticks(x_ticks, x_labels)
plt.xlabel('max_depth')
plt.ylabel('Cross-Validation Accuracy')
plt.title('Effect of max_depth on Model Performance')
plt.grid(True)
plt.show()

final_models.append(('Random Forest', best_rf_model))

### Model 2: AdaBoost

In [ ]:
# Parameters to tune
rates = [0.001, 0.01, 0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0]
max_depths = [1, 2, 3, 4, 5]

# Initialize tracking variables
results = {}
best_params = {"rate": 0, "depth": 0}
best_accuracy = 0
best_auc = 0

# Perform grid search
for depth in max_depths:
    results[depth] = {"rates": [], "accuracies": [], "aucs": [], "roc_curves": []}

    for rate in rates:
        # Create and train model
        ab_clf = AdaBoostClassifier(
            DecisionTreeClassifier(max_depth=depth, random_state=42),
            n_estimators=50,
            learning_rate=rate,
            random_state=42,
        )
        ab_clf.fit(X_train, y_train)

        # Get predictions
        ab_y_pred = ab_clf.predict(X_test)
        ab_y_prob = ab_clf.predict_proba(X_test)[:, 1]

        # Calculate metrics
        accuracy = accuracy_score(y_test, ab_y_pred)
        fpr, tpr, _ = roc_curve(y_test, ab_y_prob)
        auc = roc_auc_score(y_test, ab_y_prob)

        # Store results
        results[depth]["rates"].append(rate)
        results[depth]["accuracies"].append(accuracy)
        results[depth]["aucs"].append(auc)
        results[depth]["roc_curves"].append((fpr, tpr))

        # Update best params if needed
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params["rate"] = rate
            best_params["depth"] = depth

        if auc > best_auc:
            best_auc = auc

# Print results
print(f"Best parameters: learning_rate={best_params['rate']}, max_depth={best_params['depth']}")
print(f"Best accuracy: {best_accuracy:.4f}")

# Visualize accuracy results
plt.figure(figsize=(15, 10))

for depth in max_depths:
    plt.plot(rates, results[depth]["accuracies"], marker='o', label=f'max_depth={depth}')

plt.xscale('log')
plt.xlabel('Learning Rate')
plt.ylabel('Accuracy')
plt.title('AdaBoost Accuracy vs Learning Rate for Different max_depth Values')
plt.legend()
plt.grid(True)

# Visualize ROC curves for best parameters for each max_depth
plt.figure(figsize=(12, 10))

for depth in max_depths:
    best_idx = np.argmax(results[depth]["accuracies"])
    best_rate = results[depth]["rates"][best_idx]
    fpr, tpr = results[depth]["roc_curves"][best_idx]
    auc = results[depth]["aucs"][best_idx]
    plt.plot(fpr, tpr, label=f'max_depth={depth}, rate={best_rate}, AUC={auc:.4f}')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Best Parameters')
plt.legend()

# Create heatmap of accuracies
plt.figure(figsize=(12, 8))
heatmap_data = np.zeros((len(max_depths), len(rates)))

for i, depth in enumerate(max_depths):
    for j, rate in enumerate(rates):
        idx = results[depth]["rates"].index(rate)
        heatmap_data[i, j] = results[depth]["accuracies"][idx]

plt.imshow(heatmap_data, cmap='viridis', aspect='auto')
plt.colorbar(label='Accuracy')
plt.xticks(np.arange(len(rates)), [str(rate) for rate in rates], rotation=45)
plt.yticks(np.arange(len(max_depths)), [str(depth) for depth in max_depths])
plt.xlabel('Learning Rate')
plt.ylabel('Max Depth')
plt.title('Accuracy Heatmap')

for i in range(len(max_depths)):
    for j in range(len(rates)):
        plt.text(j, i, f"{heatmap_data[i, j]:.4f}",
                 ha="center", va="center", color="white" if heatmap_data[i, j] < 0.8 else "black")

plt.tight_layout()
plt.show()

ab_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=best_params['depth'], random_state=42),
    n_estimators=500,
    learning_rate=best_params['rate'],
    random_state=42
)
ab_clf.fit(X_train, y_train)
ab_y_prob = ab_clf.predict_proba(X_test)[:, 1]
ab_fpr, ab_tpr, _ = roc_curve(y_test, ab_y_prob)
ab_auc = roc_auc_score(y_test, ab_y_prob)
final_models.append(('AdaBoost', ab_clf))

### Model 3: Bagging

In [ ]:
# Parameters to tune
max_depths = [1, 2, 3, 4, 5, 10, None]  # None means nodes are expanded until all leaves are pure
n_estimators_list = [50, 150, 500]
bootstrap_options = [True, False]
max_samples_list = [0.5, 0.7, 1.0]  # Proportion of samples to draw

# Initialize tracking variables
results = {}
best_params = {
    "max_depth": 0,
    "n_estimators": 0,
    "bootstrap": False,
    "max_samples": 0
}
best_accuracy = 0
best_auc = 0

# Create a dictionary to store all results for visualization
all_results = []

# Perform grid search
for max_depth in max_depths:
    print('*' * 20)
    print(f'Trying max depth {max_depth}')
    for n_estimators in n_estimators_list:
        print(f'Trying {n_estimators=}')
        for bootstrap in bootstrap_options:
            print(f'Trying {bootstrap=}')
            for max_samples in max_samples_list:
                # Skip invalid combinations (max_samples only applicable with bootstrap=True)

                if not bootstrap and max_samples != 1.0:
                    continue
                print(f'Trying {max_samples=}')

                # Create base estimator
                base_estimator = DecisionTreeClassifier(max_depth=max_depth, random_state=42)

                # Create and train model
                bag_clf = BaggingClassifier(
                    base_estimator,
                    n_estimators=n_estimators,
                    bootstrap=bootstrap,
                    max_samples=max_samples if bootstrap else 1.0,
                    random_state=42,
                    n_jobs=-1
                )
                bag_clf.fit(X_train, y_train)

                # Get predictions
                bag_y_pred = bag_clf.predict(X_test)
                bag_y_prob = bag_clf.predict_proba(X_test)[:, 1]

                # Calculate metrics
                accuracy = accuracy_score(y_test, bag_y_pred)
                try:
                    fpr, tpr, _ = roc_curve(y_test, bag_y_prob)
                    auc = roc_auc_score(y_test, bag_y_prob)
                except:
                    fpr, tpr = [0, 1], [0, 1]
                    auc = 0.5

                # Store results
                config = {
                    "max_depth": max_depth,
                    "n_estimators": n_estimators,
                    "bootstrap": bootstrap,
                    "max_samples": max_samples,
                    "accuracy": accuracy,
                    "auc": auc,
                    "roc_curve": (fpr, tpr)
                }
                all_results.append(config)

                # Update best params if needed
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {
                        "max_depth": max_depth,
                        "n_estimators": n_estimators,
                        "bootstrap": bootstrap,
                        "max_samples": max_samples
                    }

                if auc > best_auc:
                    best_auc = auc

# Print best results
print(f"Best parameters:")
print(f"  max_depth: {best_params['max_depth']}")
print(f"  n_estimators: {best_params['n_estimators']}")
print(f"  bootstrap: {best_params['bootstrap']}")
print(f"  max_samples: {best_params['max_samples']}")
print(f"Best accuracy: {best_accuracy:.4f}")

# Visualize effect of max_depth with other parameters fixed
plt.figure(figsize=(15, 10))

# Filter results for bootstrap=True, n_estimators=50, max_samples=1.0
filtered_results = [r for r in all_results
                   if r["bootstrap"] == True and
                      r["n_estimators"] == 50 and
                      r["max_samples"] == 1.0]

# Sort by max_depth for visualization
filtered_results.sort(key=lambda x: str(x["max_depth"]))  # Convert None to string for sorting

depth_values = [r["max_depth"] for r in filtered_results]
depth_labels = [str(d) for d in depth_values]
accuracies = [r["accuracy"] for r in filtered_results]

plt.bar(depth_labels, accuracies)
plt.xlabel('Max Depth')
plt.ylabel('Accuracy')
plt.title('Effect of Max Depth on Bagging Classifier Accuracy')
plt.ylim(min(accuracies) - 0.05, max(accuracies) + 0.05)

# Visualize effect of n_estimators
plt.figure(figsize=(15, 10))

# Filter results for bootstrap=True, max_depth=3, max_samples=1.0
filtered_results = [r for r in all_results
                   if r["bootstrap"] == True and
                      r["max_depth"] == 3 and
                      r["max_samples"] == 1.0]

# Sort by n_estimators
filtered_results.sort(key=lambda x: x["n_estimators"])

n_estimators_values = [r["n_estimators"] for r in filtered_results]
accuracies = [r["accuracy"] for r in filtered_results]

plt.plot(n_estimators_values, accuracies, marker='o')
plt.xlabel('Number of Estimators')
plt.ylabel('Accuracy')
plt.title('Effect of Number of Estimators on Bagging Classifier Accuracy')

# Visualize bootstrap vs no bootstrap
plt.figure(figsize=(15, 10))

# Filter results for max_depth=3, n_estimators=50, max_samples=1.0
bootstrap_true = [r for r in all_results
                 if r["max_depth"] == 3 and
                    r["n_estimators"] == 50 and
                    r["max_samples"] == 1.0 and
                    r["bootstrap"] == True]

bootstrap_false = [r for r in all_results
                  if r["max_depth"] == 3 and
                     r["n_estimators"] == 50 and
                     r["max_samples"] == 1.0 and
                     r["bootstrap"] == False]

labels = ['Bootstrap', 'No Bootstrap']
values = [bootstrap_true[0]["accuracy"] if bootstrap_true else 0,
          bootstrap_false[0]["accuracy"] if bootstrap_false else 0]

plt.bar(labels, values)
plt.ylabel('Accuracy')
plt.title('Effect of Bootstrap on Bagging Classifier Accuracy')
plt.ylim(min(values) - 0.05, max(values) + 0.05)

# Visualize effect of max_samples with bootstrap=True
plt.figure(figsize=(15, 10))

# Filter results for bootstrap=True, max_depth=3, n_estimators=50
filtered_results = [r for r in all_results
                   if r["bootstrap"] == True and
                      r["max_depth"] == 3 and
                      r["n_estimators"] == 50]

# Sort by max_samples
filtered_results.sort(key=lambda x: x["max_samples"])

max_samples_values = [r["max_samples"] for r in filtered_results]
accuracies = [r["accuracy"] for r in filtered_results]

plt.plot(max_samples_values, accuracies, marker='o')
plt.xlabel('Max Samples')
plt.ylabel('Accuracy')
plt.title('Effect of Max Samples on Bagging Classifier Accuracy')

# Create a summary of top 10 configurations
all_results.sort(key=lambda x: x["accuracy"], reverse=True)
top_configs = all_results[:10]

plt.figure(figsize=(15, 10))
config_labels = [f"D:{r['max_depth']},N:{r['n_estimators']},\nB:{r['bootstrap']},S:{r['max_samples']}" for r in top_configs]
accuracies = [r["accuracy"] for r in top_configs]

plt.bar(range(len(config_labels)), accuracies)
plt.xticks(range(len(config_labels)), config_labels, rotation=45)
plt.xlabel('Configuration')
plt.ylabel('Accuracy')
plt.title('Top 10 Bagging Classifier Configurations')

# Visualize ROC curves for top 5 configurations
plt.figure(figsize=(12, 10))

for i, config in enumerate(top_configs[:5]):
    fpr, tpr = config["roc_curve"]
    plt.plot(fpr, tpr,
             label=f"D:{config['max_depth']},N:{config['n_estimators']},B:{config['bootstrap']},S:{config['max_samples']}, AUC:{config['auc']:.4f}")

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Top 5 Configurations')
plt.legend()

plt.tight_layout()
plt.show()

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_depth=None, random_state=42),
    n_estimators=500,
    bootstrap=True,
    n_jobs=-1,
    max_samples=0.5,
    random_state=40
)
bag_clf.fit(X_train, y_train)
bag_y_prob = bag_clf.predict_proba(X_test)[:, 1]
bag_fpr, bag_tpr, _ = roc_curve(y_test, bag_y_prob)
bag_auc = roc_auc_score(y_test, bag_y_prob)

### Model 4: Support Vector Machines

In [ ]:
from sklearn.metrics import auc as auc_score
from perform_kickstarer_eda import X_train, X_test, y_train, y_test
# TODO: REMOVE, ITS ALREADY ABOVEW

def evaluate_svc_models_rapids(models, X, y):
    results = []
    names = []

    # Convert to GPU if needed (assuming X and y are numpy arrays)
    X_gpu = cudf.DataFrame(X) if not isinstance(X, cudf.DataFrame) else X
    y_gpu = cudf.Series(y) if not isinstance(y, cudf.Series) else y

    for name, _ in models:
        print(f"Tuning hyperparameters for: {name}")
        kfold = KFold(n_splits=10, shuffle=True, random_state=42)

        if name == 'SVC_RBF':
            param_grid = {
                'C': [0.1, 1, 10],
                'gamma': ['scale', 'auto', 0.01, 0.1]
            }
            # cuML SVC doesn't use 'probability' parameter like sklearn
            model = cuSVC(kernel='rbf')
        elif name == 'SVC_POLY':
            param_grid = {
                'C': [0.1, 1, 10],
                'degree': [2, 3, 4],
                'gamma': ['scale', 'auto']
            }
            model = cuSVC(kernel='poly')
        elif name == 'SVC_LINEAR':
            param_grid = {
                'C': [0.1, 1, 10],
                'tol': [1e-4, 1e-3]
            }
            model = cuSVC(kernel='linear')
        elif name == 'SVC_SIGMOID':
            param_grid = {
                'C': [0.1, 1, 10],
                'gamma': ['scale', 'auto']
            }
            model = cuSVC(kernel='sigmoid')
        else:
            continue

        # Manual grid search for cuML SVC (since GridSearchCV isn't directly compatible)
        best_score = 0
        best_params = {}
        best_model = None

        # Generate parameter combinations
        param_combinations = []
        for param_name, param_values in param_grid.items():
            if not param_combinations:
                param_combinations = [{param_name: value} for value in param_values]
            else:
                new_combinations = []
                for combo in param_combinations:
                    for value in param_values:
                        new_combo = combo.copy()
                        new_combo[param_name] = value
                        new_combinations.append(new_combo)
                param_combinations = new_combinations

        # Perform manual grid search with cross-validation
        for params in param_combinations:
            current_model = cuSVC(kernel=model.kernel, **params)
            fold_scores = []

            for train_idx, val_idx in kfold.split(X):
                # Extract folds (using numpy indices works with cuDF)
                X_train_fold = X_gpu.iloc[train_idx]
                y_train_fold = y_gpu.iloc[train_idx]
                X_val_fold = X_gpu.iloc[val_idx]
                y_val_fold = y_gpu.iloc[val_idx]

                # Train and evaluate
                current_model.fit(X_train_fold, y_train_fold)
                y_pred = current_model.predict(X_val_fold)
                score = accuracy_score(y_val_fold, y_pred)
                fold_scores.append(score)

            # Calculate mean score across folds
            mean_score = sum(fold_scores) / len(fold_scores)

            # Update best model if this one is better
            if mean_score > best_score:
                best_score = mean_score
                best_params = params
                best_model = current_model

        # Train best model on full dataset
        best_model = cuSVC(kernel=model.kernel, **best_params)
        best_model.fit(X_gpu, y_gpu)

        print(f"Best params for {name}: {best_params}")
        print(f"Best CV accuracy: {best_score:.4f}")

        # For metrics, convert predictions back to numpy for sklearn compatibility
        # Manual cross-validation for F1 score
        y_pred_list = []
        y_true_list = []

        # Temporary workaround for ROC since cuML SVC doesn't have predict_proba
        # We'll use decision_function instead which gives distance from hyperplane
        decision_scores_list = []

        for train_idx, val_idx in kfold.split(X):
            X_train_fold = X_gpu.iloc[train_idx]
            y_train_fold = y_gpu.iloc[train_idx]
            X_val_fold = X_gpu.iloc[val_idx]
            y_val_fold = y_gpu.iloc[val_idx]

            fold_model = cuSVC(kernel=model.kernel, **best_params)
            fold_model.fit(X_train_fold, y_train_fold)

            y_pred = fold_model.predict(X_val_fold)

            # Use decision_function instead of predict_proba for ROC curve
            decision_scores = fold_model.decision_function(X_val_fold)

            # Handle different return types (cuDF Series/DataFrame or numpy array)
            if hasattr(y_pred, 'to_numpy'):
                y_pred_list.extend(y_pred.to_numpy())
            else:
                y_pred_list.extend(y_pred)

            if hasattr(y_val_fold, 'to_numpy'):
                y_true_list.extend(y_val_fold.to_numpy())
            else:
                y_true_list.extend(y_val_fold)

            if hasattr(decision_scores, 'to_numpy'):
                decision_scores_list.extend(decision_scores.to_numpy())
            else:
                decision_scores_list.extend(decision_scores)

        # Calculate F1 score
        f1 = f1_score(np.array(y_true_list), np.array(y_pred_list))
        print(f"F1 Score for {name}: {f1:.4f}")

        # ROC Curve using decision function scores instead of probabilities
        fpr, tpr, _ = roc_curve(np.array(y_true_list), np.array(decision_scores_list))
        roc_auc = auc_score(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{name} (AUC = {roc_auc:.2f})")

        # Save results for boxplot
        cv_results = np.array(fold_scores)
        results.append(cv_results)
        names.append(name)

        print(f"F1 Score: {f1:.4f} | AUC: {roc_auc:.4f}")

    # Plot results
    fig = plt.figure()
    fig.suptitle('Tuned cuML SVC Model Comparison')
    ax = fig.add_subplot(111)
    plt.boxplot(results, labels=names)
    ax.set_ylabel('Accuracy')
    plt.show()

    return results, names

# Define models
models = [('SVC_RBF', None), ('SVC_POLY', None), ('SVC_LINEAR', None), ('SVC_SIGMOID', None)]

# Run evaluation
evaluate_svc_models_rapids(models, X_train, y_train)

### Model 5: Multilayer Perceptron

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
configs = [
    {"layers": [16], "activation": "relu", "optimizer": "adam", "learning_rate": 0.001},
    {"layers": [16], "activation": "relu", "optimizer": "adam", "learning_rate": 0.01},
    {"layers": [32], "activation": "relu", "optimizer": "adam", "learning_rate": 0.001},
    {"layers": [32], "activation": "relu", "optimizer": "adam", "learning_rate": 0.01},
    {"layers": [64], "activation": "relu", "optimizer": "adam", "learning_rate": 0.001},
    {"layers": [64], "activation": "relu", "optimizer": "adam", "learning_rate": 0.01},
    {"layers": [128], "activation": "relu", "optimizer": "adam", "learning_rate": 0.001},
    {"layers": [128], "activation": "relu", "optimizer": "adam", "learning_rate": 0.01},
    {"layers": [256], "activation": "relu", "optimizer": "adam", "learning_rate": 0.001},
    {"layers": [256], "activation": "relu", "optimizer": "adam", "learning_rate": 0.01},
    {"layers": [32, 16], "activation": "relu", "optimizer": "adam", "learning_rate": 0.001},
    {"layers": [32, 16], "activation": "relu", "optimizer": "adam", "learning_rate": 0.01},
    {"layers": [64, 32], "activation": "relu", "optimizer": "adam", "learning_rate": 0.001},
    {"layers": [64, 32], "activation": "relu", "optimizer": "adam", "learning_rate": 0.01},
    {"layers": [128, 64], "activation": "relu", "optimizer": "adam", "learning_rate": 0.001},
    {"layers": [128, 64], "activation": "relu", "optimizer": "adam", "learning_rate": 0.01},
    {"layers": [64, 32, 16], "activation": "relu", "optimizer": "adam", "learning_rate": 0.001},
    {"layers": [64, 32, 16], "activation": "relu", "optimizer": "adam", "learning_rate": 0.01},

    {"layers": [16], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.001},
    {"layers": [16], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.01},
    {"layers": [32], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.001},
    {"layers": [32], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.01},
    {"layers": [64], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.001},
    {"layers": [64], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.01},
    {"layers": [128], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.001},
    {"layers": [128], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.01},
    {"layers": [256], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.001},
    {"layers": [256], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.01},
    {"layers": [32, 16], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.001},
    {"layers": [32, 16], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.01},
    {"layers": [64, 32], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.001},
    {"layers": [64, 32], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.01},
    {"layers": [128, 64], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.001},
    {"layers": [128, 64], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.01},
    {"layers": [64, 32, 16], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.001},
    {"layers": [64, 32, 16], "activation": "relu", "optimizer": "sgd", "learning_rate": 0.01}
]

results = []
for config in configs:
    print(f"Trying config: {config}")
    tf.keras.backend.clear_session()

    if config["optimizer"] == "adam":
        optimizer = tf.keras.optimizers.Adam(config["learning_rate"])
    elif config["optimizer"] == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=config["learning_rate"])

    # Build model with current configuration
    input_layer = tf.keras.layers.Input(shape=(X_train.shape[1],))
    x = input_layer
    for units in config["layers"]:
        x = tf.keras.layers.Dense(units, activation=config["activation"])(x)

    # Binary classification output
    output = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.models.Model(inputs=input_layer, outputs=output)
    model.compile(
        loss="binary_crossentropy",
        optimizer=optimizer,
        metrics=["accuracy", tf.keras.metrics.AUC()]
    )
    model.summary()

    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=50,
        validation_data=(X_test, y_test),
        verbose=0
    )

    # Get best results based on validation accuracy
    val_acc_hist = history.history["val_accuracy"]
    best_val_acc = np.max(val_acc_hist)
    best_epoch = np.argmax(val_acc_hist)

    best_loss = history.history["loss"][best_epoch]
    best_acc = history.history["accuracy"][best_epoch]
    best_val_loss = history.history["val_loss"][best_epoch]
    best_auc = history.history["auc"][best_epoch]
    best_val_auc = history.history["val_auc"][best_epoch]

    print(f"Best epoch: {best_epoch+1} | loss: {best_loss:.4f} - acc: {best_acc:.4f} - auc: {best_auc:.4f} - val_loss: {best_val_loss:.4f} - val_acc: {best_val_acc:.4f} - val_auc: {best_val_auc:.4f}")

    # Store results
    results.append({
        "config": config,
        "epoch": best_epoch+1,
        "loss": best_loss,
        "accuracy": best_acc,
        "auc": best_auc,
        "val_loss": best_val_loss,
        "val_accuracy": best_val_acc,
        "val_auc": best_val_auc,
        "history": history.history
    })

# Display results table
df_results = DataFrame(results)
display(df_results)

# Select the best model configuration based on validation accuracy
best_idx = df_results["val_accuracy"].idxmax()
best_config = df_results.loc[best_idx, "config"]
print(f"Best configuration: {best_config}")

# Build final model with best configuration
tf.keras.backend.clear_session()
input_layer = tf.keras.layers.Input(shape=(X_train.shape[1],))
x = input_layer
for units in best_config["layers"]:
    x = tf.keras.layers.Dense(units, activation=best_config["activation"])(x)
output = tf.keras.layers.Dense(1, activation="sigmoid")(x)

final_model = tf.keras.models.Model(inputs=input_layer, outputs=output)
final_model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy", tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Train the final model
final_history = final_model.fit(
    X_train, y_train,
    epochs=50,
    validation_data=(X_test, y_test),
    verbose=1
)
# Plot learning curves for final model
DataFrame(final_history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.title("Learning Curves")
plt.xlabel("Epoch")
plt.ylabel("Value")
plt.legend()
plt.tight_layout()
plt.show()
# Evaluate the final model
test_loss, test_acc, test_auc, test_precision, test_recall = final_model.evaluate(X_test, y_test)

# Get predictions
y_pred_prob = final_model.predict(X_test).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

# Print metrics
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test AUC: {test_auc:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Plot ROC curve
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc_score(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

final_models.append(('MLP', final_model))

### Model 6: Decision Tree

In [ ]:
# Get the optimal features we found
optimal_features = ['backers_count', 'goal', 'launch_to_state_change_days', 'create_to_launch_days', 'launch_to_deadline_days', 'created_at_day', 'blurb_len_clean', 'created_at_hr', 'launched_at_day', 'name_len_clean', 'launched_at_hr', 'state_changed_at_day', 'deadline_day', 'created_at_month', 'state_changed_at_hr', 'deadline_hr', 'launched_at_month', 'state_changed_at_month', 'category_Gadgets', 'deadline_month', 'category_Software', 'category_Musical', 'category_Hardware', 'category_Apps', 'category_None', 'category_Web', 'created_at_weekday_Friday', 'staff_pick', 'created_at_weekday_Wednesday', 'deadline_weekday_Thursday']

# Train the model using optimal features
X_train_optimal = X_train[optimal_features]
X_test_optimal = X_test[optimal_features]

# Define parameter grid for hyperparameter tuning
param_grid = {
    'max_depth': range(1, 21, 2),
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X_train_optimal, y_train)

# Get best parameters and model
best_params = grid_search.best_params_
best_dt_model = grid_search.best_estimator_

print("Best parameters:", grid_search.best_params_)
y_pred = best_dt_model.predict(X_test_optimal)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Evaluate classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Evaluate confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Feature importance of the final model
feature_names = best_dt_model.feature_importances_
importance_df = DataFrame({
    'Feature': optimal_features,
    'Importance': feature_names
}).sort_values('Importance', ascending=False)

print("\nFeature Importance:")
print(importance_df.head(10))  # Show top 10 features

# Calculate cross-validation score for the best model
cv_scores = cross_val_score(
    best_dt_model,
    X[optimal_features], Y1,
    cv=5, scoring='accuracy'
)
print(f"\nCross-validation accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
plt.figure(figsize=(10, 6))
plot_tree(best_dt_model, filled=True,
          feature_names=feature_names,
          precision=2, proportion=True)
plt.title(f"Classifier Decision Tree: Depth {best_dt_model.get_depth()}")
plt.tight_layout()
plt.show()
# Plot ROC curve
y_scores = best_dt_model.predict_proba(X_test_optimal)[:, 1]
fpr, tpr, thresholds2 = roc_curve(y_test, y_scores)
roc_auc = auc_score(fpr, tpr)

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2,
         label=f'ROC curve (area = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Visualize hyperparameter tuning results
results = pd.DataFrame(grid_search.cv_results_)
plt.figure(figsize=(12, 6))

# Filter results for default min_samples values
default_samples = results[(results['param_min_samples_split'] == 2) &
                         (results['param_min_samples_leaf'] == 1)]

# Convert None to -1 for plotting purposes
default_samples['param_max_depth'] = default_samples['param_max_depth'].fillna(-1)

# Sort by max_depth for proper x-axis order
default_samples = default_samples.sort_values('param_max_depth')

# Plot the results
plt.plot(default_samples['param_max_depth'],
         default_samples['mean_test_score'],
         marker='o', linestyle='-')

# Replace -1 with "None" in xticks
x_ticks = default_samples['param_max_depth'].unique()
x_labels = [str(int(x)) if x > 0 else "None" for x in x_ticks]
plt.xticks(x_ticks, x_labels)
plt.xlabel('max_depth')
plt.ylabel('Cross-Validation Accuracy')
plt.title('Effect of max_depth on Decision Tree Performance')
plt.grid(True)
plt.show()

final_models.append(('Decision Tree', best_dt_model))

# Part 3: Display results (from all final models)

In [ ]:
for name, model in final_models:
    print(f"Model: {name}")
    # TODO: Display results for each model